In [2]:
import sys
print(sys.executable)

/usr/local/bin/python3


In [1]:
pip install numpy torch ultralytics roboflow albumentations matplotlib

Note: you may need to restart the kernel to use updated packages.


In [7]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.163 🚀 Python-3.13.5 torch-2.7.1 CPU (Apple M3 Pro)
Setup complete ✅ (11 CPUs, 18.0 GB RAM, 319.3/460.4 GB disk)


In [6]:
from roboflow import Roboflow
rf = Roboflow(api_key="ug2qlXusJiZoRmRSug6C")
project = rf.workspace("umat").project("pothole-abpqz")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [37]:
import torch
import numpy as np
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

In [4]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [9]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print (device)

mps


In [ ]:
model = YOLO('yolov8m-seg.pt')
results = model.train(data = f"{dataset.location}/data.yaml",
                                epochs = 100,
                                device = device,
                                name = 'pothole_detection',
                                optimizer = 'Adam',
                                lr0 = 0.01,
                                imgsz = 640,
                                mask_ratio = 2,
                                overlap_mask = True,
                                augment = True,
                                batch = -1
                                 )

In [77]:
model = YOLO("./runs_2/segment/pothole_detection3/weights/best.pt")

metrics = model.val( data = f"{dataset.location}/data.yaml",
                     split = "test",
                     device = device,
                     conf = 0.5,
                     iou = 0.6,
                     plots = True
                     )

print(f"mAP@50-90: {metrics.box.map}")
print(f"mask-accuracy: {metrics.seg.map}")

YOLOv8m-seg summary (fused): 105 layers, 27,222,963 parameters, 0 gradients, 110.0 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 334.0±107.0 MB/s, size: 58.2 KB)


val: Scanning /Users/pawankalyanreddy/Projects/ML_Notebooks/Pothole-1/test/labels.cache... 55 images, 0 backgrounds, 0 corrupt: 100%|██████████| 55/55 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.24it/s]


                   all         55        193       0.84      0.627      0.734      0.618       0.84      0.627      0.737      0.564
Speed: 0.5ms preprocess, 29.2ms inference, 0.0ms loss, 15.7ms postprocess per image
Results saved to runs/segment/val4
mAP@50-90: 0.6175576615738968
mask-accuracy: 0.564424543740855


In [3]:
import gc

gc.collect()
torch.mps.empty_cache()

In [103]:
#from google.colab.patches import cv2_imshow
def image_processing(image_path, model_path="./runs/segment/pothole_detection2/weights/best.pt"):
    image = cv2.imread(image_path)
    model = YOLO(model_path)
    original = image.copy()

    results = model(image)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        for box in boxes:
            x1,y1,x2,y2 = map(int, box)
            cv2.rectangle(image, (x1,y1), (x2,y2), (0,255,0), 3)

        if result.masks is not None:
            masks = result.masks.data.cpu().numpy()
            for mask in masks:
                mask = (mask>0.5).astype(np.uint8)*255

                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                overlay = original.copy()
                for cnt in contours:
                    cv2.drawContours(overlay, contours, -1, (0,0,255), -1)
                    cv2.drawContours(image, [cnt], -1, (255,0,0), 2 )
            
                # overlay = original.copy()
                # cv2.drawContours(overlay, contours, -1, (0,0,255), -1)
                image = cv2.addWeighted(overlay, 0.3, image, 0.7, 0)


    #cv2_imshow(image)
    return image



In [104]:
result = image_processing("/Users/pawankalyanreddy/Projects/ML_Notebooks/Pothole-1/test/istockphoto-2138539780-612x612.jpg")
cv2.imwrite("annotated.jpg", result)


0: 480x640 2 potholes, 149.4ms
Speed: 1.1ms preprocess, 149.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)


True

In [82]:
import cv2 
def video_processing(video_path, output_path = "output.mp4" ):
    model = YOLO("./runs/segment/pothole_detection2/weights/best.pt")
    cap = cv2.VideoCapture(video_path)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        results = model(frame)

        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            for box in boxes:
                x1,y1,x2,y2 = map(int, box)
                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)

            if result:
                masks = result.masks.data.cpu().numpy()
                for mask in masks:
                    mask = (mask>0.5).astype(np.uint8)
                    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 

                    for cnt in contours:
                        cv2.drawContours(frame, [cnt], -1, (255,0,0), 2)

                    overlay = frame.copy()
                    cv2.drawContours(frame, contours, -1, (0,0,225), -1)
                    cv2.addWeighted(overlay, 0.3, frame, 0.7, 0)

        out.write(frame)
    cap.release()
    out.release()
    return output_path


In [80]:
result1 = video_processing("/Users/pawankalyanreddy/Projects/ML_Notebooks/Pothole-1/087888615_main_xxl.mp4", )


0: 384x640 (no detections), 151.7ms
Speed: 1.0ms preprocess, 151.7ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 120.6ms
Speed: 0.9ms preprocess, 120.6ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 122.6ms
Speed: 0.8ms preprocess, 122.6ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 126.6ms
Speed: 0.9ms preprocess, 126.6ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 127.3ms
Speed: 0.9ms preprocess, 127.3ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 126.4ms
Speed: 0.9ms preprocess, 126.4ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 128.0ms
Speed: 0.9ms preprocess, 128.0ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 130.1ms
Speed: 0.8ms prepr

In [83]:
result2 = video_processing("/Users/pawankalyanreddy/Projects/ML_Notebooks/Pothole-1/mixkit-potholes-in-a-rural-road-25208-hd-ready.mp4")


0: 384x640 (no detections), 138.1ms
Speed: 1.2ms preprocess, 138.1ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 122.0ms
Speed: 0.8ms preprocess, 122.0ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pothole, 121.1ms
Speed: 0.8ms preprocess, 121.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 119.6ms
Speed: 0.7ms preprocess, 119.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 123.9ms
Speed: 0.8ms preprocess, 123.9ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 123.0ms
Speed: 0.8ms preprocess, 123.0ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 125.5ms
Speed: 0.8ms preprocess, 125.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pothole, 125.7ms
Speed: 1.1ms preprocess, 125.7